# 优化算法

## 1. 微批梯度下降

批量梯度下降一次性将所有样本进行矩阵运算，这个向量化运算的过程，利用CPU或GPU的并行计算指令集，可以获得比单个样本计算更高的效率。

当样本量过大时，比如来到百万级或以上，视内存或显存的限制而定，对超大的矩阵进行运算，反而会导致效率下降。微批梯度下降的思路，就是将训练样本分组，对每个组（每个批次）进行梯度下降运算。

对每个批次的一次前向传播和反向传播计算过程，称为一个**阶段 epoch**。而将所有批次进行计算完成后，参数还不会收敛。需要持续循环。

![Mini-batch gradient descent](img/Mini-batch gradient descent.png)